In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display, Markdown

# 1. The New Way - Docker Deploy

To start, ensure you have Docker installed and running!

In your terminal, run:
```
docker desktop enable model-runner --tcp=12434
```

Then run 
```
docker model pull ai/ai/deepseek-r1-distill-llama:8B-Q4_K_M
```
to pull a quantized deepseek-r1 distilled llama 8b model.

In [3]:
docker_url = "http://localhost:12434/engines/v1"
docker_llm = "ai/deepseek-r1-distill-llama:8B-Q4_K_M"

#### Testing the Docker deployed model - with OpenAI!

In [4]:
from openai import OpenAI

docker_client = OpenAI(base_url=docker_url, api_key="fake")

prompt = "Why is the sky blue?"
response = docker_client.chat.completions.create(
    model=docker_llm,
    messages = [
        {"role": "user", "content": prompt}
    ]
)

In [5]:
display(Markdown(response.choices[0].message.content))

The sky appears blue primarily due to Rayleigh scattering, where shorter wavelengths of light, such as blue, are scattered more efficiently by gas molecules in the atmosphere. Here's a concise explanation:

1. **Rayleigh Scattering**: This process causes light with shorter wavelengths (blue) to scatter more than longer wavelengths (red, orange). The scattering intensity is inversely proportional to the square of the wavelength, meaning blue light is scattered more.

2. **Scattered Light**: The scattered blue light comes from all directions, but we only perceive the light that scatters towards us, creating the blue appearance of the sky.

3. **Sunrise and Sunset**: During these times, the sun's light travels through more of the atmosphere, scattering away more blue light, allowing longer wavelengths (red and orange) to dominate the sky's appearance.

4. **Human Perception**: Our eyes are more sensitive to blue, making it the most noticeable color in the sky.

5. **Altitude and Atmosphere**: At higher altitudes, less atmosphere means less scattering, potentially making the sky appear black. On other planets, similar atmospheric conditions can result in blue skies, though other factors like atmospheric composition may alter the color.

In summary, Rayleigh scattering is the primary reason for the blue sky during the day, with other factors influencing color changes like sunrise or sunset.

#### Using LangChain

In [6]:
from langchain_openai import ChatOpenAI

langchain_llm = ChatOpenAI(
    model_name = docker_llm,
    temperature = 0,
    api_key = "fake",
    openai_api_base = docker_url
)

response = langchain_llm.invoke([{"role": "user", "content": prompt}])

In [7]:
display(Markdown(response.content))

The sky appears blue due to a phenomenon known as Rayleigh scattering. Here's a concise explanation of the process:

1. **Rayleigh Scattering**: This occurs when light scatters off small particles in the Earth's atmosphere. The scattering is more efficient for shorter wavelengths of light, such as blue and violet.

2. **Forward Scattering**: The scattered light is directed in a manner that is close to the original direction of travel. This means that the scattered blue light we see is coming towards us, contributing to the blue appearance of the sky.

3. **Atmospheric Density**: The Earth's atmosphere is denser near the surface, leading to more scattering of blue light. This results in the sky appearing blue to observers on the ground.

4. **Sunrise and Sunset**: During these times, the sun's light passes through a greater thickness of the atmosphere, scattering away blue and violet light, allowing red light to dominate and giving the sky its red hue.

5. **Black Sky in Space**: In the absence of an atmosphere, as experienced by astronauts, the sky appears black because there is no scattering of light.

In summary, the blue color of the sky is primarily due to Rayleigh scattering of blue light, which is efficiently scattered and directed towards us, combined with the Earth's atmospheric density near the surface.

#### Using LlamaIndex

In [8]:
import nest_asyncio
from llama_index.llms.openai_like import OpenAILike

nest_asyncio.apply()

llm = OpenAILike(
    model = docker_llm,
    api_base = docker_url,
    api_key = "fake",
    is_chat_model = True,
    is_function_calling_model = True,
    temperature = 0
)

response = llm.complete("Hi!")

display(Markdown(str(response)))

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Hi! How can I assist you today?

Creating a tool

In [9]:
from llama_index.core.agent.workflow import AgentStream, FunctionAgent
from llama_index.core.tools import FunctionTool

def get_weather(location: str) -> str:
    """Use this tool to get the weather in a location."""
    return f"The weather in {location} is sunny."

weather_agent = FunctionAgent(
    llm = llm,
    tools = [FunctionTool.from_defaults(get_weather)]
)

In [10]:
handler = weather_agent.run("What is the weather in Singapore?")

async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)

To find the weather in Singapore, you can use the `get_weather` tool by calling it with the location "Singapore". Here's how you can do it:

```json
<tool call="get_weather" location="Singapore">
  {"weather": "Sunny with a temperature of 28°C."}
</tool call>
```

This tool call will return the current weather in Singapore, such as the temperature and whether it's sunny or not.

Deploy LlamaIndex Workflow as an MCP!

In [21]:
%%writefile ../tools/weather_mcp.py

"""
This code will automatically generate a FastMCP server that will

- Use the agent workflow class name as the tool name
- Use our custom RunEvent as the typed inputs to the tool
- Automatically use the SSE stream for streaming json dumps of the workflow event stream
"""

from llama_index.core.agent.workflow import AgentStream, FunctionAgent
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai_like import OpenAILike
from llama_index.tools.mcp.utils import workflow_as_mcp


docker_url = "http://localhost:12434/engines/v1"
docker_llm = "ai/deepseek-r1-distill-llama:8B-Q4_K_M"

llm = OpenAILike(
    model = docker_llm,
    api_base = docker_url,
    api_key = "fake",
    is_chat_model = True,
    is_function_calling_model = True,
    temperature = 0
)
def get_weather(location: str) -> str:
    """Use this tool to get the weather in a location."""
    return f"The weather in {location} is sunny."

weather_agent = FunctionAgent(
    llm = llm,
    name = "weather_agent",
    description = "An agent that can answer questions about the weather.",
    tools = [FunctionTool.from_defaults(get_weather)]
)

mcp = workflow_as_mcp(weather_agent, workflow_name = 'weather_agent', port=8000)

if __name__ == "__main__":
    mcp.run(transport="sse")

Writing ../tools/weather_mcp.py


Spin up the mcp server:
`uv run tools/weather/mcp.py`

In [41]:
# Test if tools have been deployed!

from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

mcp_client = BasicMCPClient("http://localhost:8000/sse")
mcp_tool = McpToolSpec(client=mcp_client)
tools = await mcp_tool.to_tool_list_async()
print(tools)


Testing with OpenAI-Like

In [42]:
agent = FunctionAgent(
    llm = llm,
    tools = tools
)
handler = agent.run("What is the weather in Singapore?")

async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)

To find the current weather in Singapore, you can use the weather_agent tool by calling it with the location parameter set to "Singapore". Here's how to do it:

```json
<tool call begin>weather_agent<tool call end> run_args="Singapore"
```

Testing with OpenAI

In [47]:
from dotenv import load_dotenv, find_dotenv
from llama_index.llms.openai import OpenAI

_ = load_dotenv(find_dotenv())

openai_llm = OpenAI(model="gpt-4o-mini", temperature=0)
openai_llm.complete("Hi!")

CompletionResponse(text='Hello! How can I assist you today?', additional_kwargs={'prompt_tokens': 9, 'completion_tokens': 9, 'total_tokens': 18}, raw=ChatCompletion(id='chatcmpl-BtyBk9LRwyrH16KnC4KgLrnM3ytps', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1752679104, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))), logprobs=None, delta=None)

In [48]:
agent = FunctionAgent(
    llm = openai_llm,
    tools = tools
)
handler = agent.run("What is the weather in Singapore?")

async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)

I'm currently unable to retrieve the weather information for Singapore. You might want to check a weather website or app for the latest updates. If you have any other questions or need assistance, feel free to ask!

# 2. My favorite way - Ollama!

To start, ensure that you have Ollama installed and running.

In your terminal, run:
```
ollama pull qwen2.5
```

This will pull qwen2.5:7b by default

In [11]:
ollama_url = "http://localhost:11434/v1/"
ollama_llm = "qwen3"

#### Using the OpenAI SDK

In [12]:
ollama_client = OpenAI(base_url=ollama_url, api_key="fake")

response = ollama_client.chat.completions.create(
    model=ollama_llm,
    messages = [
        {"role": "user", "content": prompt}
    ]
)

display(Markdown(response.choices[0].message.content))

<think>
Okay, the user is asking why the sky is blue. I remember that this is a classic question related to the scattering of sunlight. Let me think about the process step by step. 

First, sunlight is white, but it's actually a combination of all colors. Each color has a different wavelength. When sunlight enters Earth's atmosphere, it interacts with the molecules and small particles in the air. I think Rayleigh scattering is involved here. Rayleigh scattering refers to the scattering of light by particles much smaller than the wavelength of the light. 

So, shorter wavelengths (like blue and violet) are scattered more than longer wavelengths (like red and orange). That means blue light is scattered in all directions by the molecules in the atmosphere. But wait, why isn't the sky violet then? Because even though violet light has an even shorter wavelength and is scattered more, the sun emits less violet light than blue. Also, our eyes are more sensitive to blue light, so we perceive the sky as blue rather than violet.

But I should also mention the role of the atmosphere's composition. Nitrogen and oxygen molecules are the main contributors to this scattering. The presence of other particles, like dust or water vapor, can affect the color, but under normal conditions, the blue dominates. 

Additionally, during sunrise or sunset, the sky appears reddish because the sunlight has to pass through more atmosphere, scattering out the shorter wavelengths and leaving the longer ones. That's why the sun looks red or orange during those times.

Wait, but I should make sure I'm not confusing Rayleigh scattering with Mie scattering. Mie scattering is for larger particles, like water droplets or aerosols, which cause more uniform scattering and can lead to white or pale colors. That's why clouds appear white, as they scatter all wavelengths of light equally.

So putting it all together, the primary reason the sky is blue is due to Rayleigh scattering of shorter wavelengths like blue and violet, combined with our eyes' sensitivity to blue light and the sun's emission spectrum. Also, the fact that violet is scattered even more but is less intense and our eyes are less sensitive to it explains why the sky isn't violet. 

I should also address possible misconceptions, like thinking the sky is blue because of reflection from the Earth's surface or something else. No, that's not the main reason. The main cause is indeed the scattering of sunlight by atmospheric molecules.
</think>

The sky appears blue due to a phenomenon called **Rayleigh scattering**, which involves the interaction of sunlight with the molecules and small particles in Earth's atmosphere. Here's a concise explanation:

### 1. **Sunlight and Color**:
   - Sunlight is composed of a spectrum of colors, each with different wavelengths (from violet/blue to red/orange).
   - Shorter wavelengths (like **blue** and **violet**) are scattered more efficiently by the atmosphere than longer wavelengths (like red or yellow).

### 2. **Rayleigh Scattering**:
   - When sunlight enters the atmosphere, it interacts with molecules like **oxygen (O₂)** and **nitrogen (N₂)**. These molecules are much smaller than the wavelength of visible light.
   - Blue/violet light (shorter wavelengths) is scattered in all directions by these molecules, while red/orange light (longer wavelengths) passes through more directly.

### 3. **Why the Sky Appears Blue, Not Violet**:
   - Although **violet** light is scattered even more than blue, the **sun emits less violet light** than blue.
   - Human eyes are **more sensitive to blue light** than violet, and the **violet light is partially absorbed by the upper atmosphere**.
   - Together, these factors make blue the dominant color we perceive.

### 4. **Additional Factors**:
   - During **sunrise or sunset**, sunlight travels through more atmosphere, scattering out the blue light and leaving the longer wavelengths (red/orange) to dominate.
   - **Clouds** and larger particles cause **Mie scattering**, which scatters all wavelengths equally, giving clouds their white or gray appearance.

### Summary:
The blue sky is a result of **Rayleigh scattering** of shorter wavelengths (blue/violet) by atmospheric molecules, combined with our eyes' sensitivity to blue light and the sun's emission spectrum. This makes blue the dominant color we see on clear days! 🌤️

#### Using the Ollama SDK

Which looks like the OpenAI SDK!

In [13]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(
    model=ollama_llm, 
    messages=[
        {"role": "user", "content": prompt}
    ]
  )

print(response['message']['content'])
# or access fields directly from the response object
# print(response.message.content)

<think>
Okay, the user is asking why the sky is blue. I remember from school that it has to do with light scattering, but I need to recall the exact details. Let me start by explaining that sunlight is white but consists of different colors. Each color has a different wavelength.

Then there's Rayleigh scattering. I think that shorter wavelengths scatter more. Blue and violet have shorter wavelengths, so they scatter more. But why is the sky blue and not violet? Maybe because our eyes are more sensitive to blue, and the sun emits more blue light than violet. Also, some of the violet light is absorbed by the upper atmosphere.

Wait, I should mention that during sunrise or sunset, the sky turns red or orange because the light has to travel through more atmosphere, scattering the shorter wavelengths out of our line of sight. That's an important point to include for a complete answer.

Let me structure this step by step. Start with the composition of sunlight, then explain Rayleigh scatter

#### Using LangChain

In [14]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model = ollama_llm,
    temperature = 0
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Chinese. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = model.invoke(messages)

In [17]:
display(Markdown(ai_msg.content))

<think>
Okay, the user wants me to translate "I love programming." from English to Chinese. Let me start by understanding the sentence. The main components are "I" (subject), "love" (verb), and "programming" (object). 

First, "I" in Chinese is "我". Then "love" is a verb, and the translation for "love" in this context would be "爱" or "喜欢". Both are correct, but "爱" is more intense, while "喜欢" is a bit more casual. Since the user said "love", maybe "爱" is better here.

Next, "programming" translates to "编程". So putting it all together: "我爱编程。" Alternatively, "我喜欢编程。" Both are correct. But the user might prefer the more direct translation. Let me check if there are any nuances. 

In Chinese, "爱" is often used for deeper affection, while "喜欢" is more about preference. Since the user says "love", using "爱" would be more accurate. So the translation should be "我爱编程。" 

Wait, but sometimes people might use "喜欢" even when they mean "love" in a general sense. Maybe the user is okay with either. But since the instruction is to translate accurately, "爱" is better. 

I should also check for any possible errors. The structure is correct: subject + verb + object. No particles needed. So the final translation is "我爱编程。"
</think>

我爱编程。

Creating our MCP agent

In [25]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

client = MultiServerMCPClient(
    {
        "weather": {
            # Make sure you start your weather server on port 8000
            "url": "http://localhost:8000/sse/",
            "transport": "sse",
        }
    }
)
tools = await client.get_tools()
agent = create_react_agent(model, tools)

In [26]:
tools

[StructuredTool(name='weather_agent', description='An agent that can answer questions about the weather.', args_schema={'$defs': {'AgentWorkflowStartEvent': {'properties': {}, 'title': 'AgentWorkflowStartEvent', 'type': 'object'}}, 'properties': {'run_args': {'$ref': '#/$defs/AgentWorkflowStartEvent'}}, 'required': ['run_args'], 'title': '_workflow_toolArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x13167f920>)]

In [32]:
async for chunk in agent.astream(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    stream_mode="updates"
):
    # chunk['messages'].pretty_print()
    print(chunk)

{'agent': {'messages': [AIMessage(content='<think>\nOkay, the user is asking for the weather in SF, which I assume stands for San Francisco. I need to use the weather_agent function to get this information. Let me check the function parameters. The function requires a run_args object, but the description for run_args is empty. Maybe the function expects certain parameters like location. Since the user mentioned SF, I should pass that as the location. Let me structure the arguments with "location": "San Francisco" to ensure the function knows where to check the weather. I\'ll format the tool call accordingly.\n</think>\n\n', additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2025-07-16T14:54:22.064369Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2815476958, 'load_duration': 33805833, 'prompt_eval_count': 148, 'prompt_eval_duration': 178164875, 'eval_count': 142, 'eval_duration': 2603170333, 'model_name': 'qwen3'}, id='run--31c2f206-443d-41fd-bb7a-4b49

CancelledError: 

In [20]:
weather_response

{'messages': [HumanMessage(content='what is the weather in nyc?', additional_kwargs={}, response_metadata={}, id='72b6831f-29d8-46ac-84d7-bf1433521f7f'),
  AIMessage(content="<think>\nOkay, the user is asking for the weather in NYC. Let me check the tools available. There's a function called FunctionAgent that requires run_args. But the description is empty, so I'm not sure what it does. Maybe it's a placeholder or a generic function. Since there's no specific weather function listed, I can't call one to get the weather data. I should inform the user that I don't have the necessary tool to retrieve the weather information. I need to mention that the available tools don't include a weather function. Alright, time to respond clearly that I can't help with that request.\n</think>\n\nThe available tools do not include a function for retrieving weather information. I cannot provide the current weather in NYC.", additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2025-0